In [5]:
import sys
sys.path.append('../')

In [6]:
from utils.functions import process_images

# Specify the folder path containing the images and annotations
folder_path = '../data/imgs/occupied'

# Specify the output folder path
output_folder = '../result/imgs/occupied_with_boxes'

process_images(folder_path, output_folder, highlighted_cars=True)

Processed 14 images


In [7]:
# Specify the folder path containing the images and annotations
folder_path = '../data/imgs/empty'

# Specify the output folder path
output_folder = '../result/imgs/empty_with_boxes'

process_images(folder_path, output_folder, highlighted_cars=True)

Processed 9 images
